# Dog Breed Identification

- 120 breeds of dogs 

- Limited number training images per class

- Multi Class Log Loss

## Imports

In [14]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import time
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

plt.ion()   # interactive mode

%matplotlib inline

In [2]:
np.random.seed(2206)

## Default Paths

In [64]:
TRAIN_IMG_PATH = "data/dog-breed-identification/train"
TEST_IMG_PATH = "data/dog-breed-identification/test"
LABELS_CSV_PATH = "data/dog-breed-identification/labels.csv"
SAMPLE_SUB_PATH = "data/dog-breed-identification/sample_submission.csv"

## Import Data

In [65]:
INPUT_SIZE = 224
NUM_CLASSES = 2

labels = pd.read_csv(LABELS_CSV_PATH)
sample_submission = pd.read_csv(SAMPLE_SUB_PATH)

print(len(os.listdir(TRAIN_IMG_PATH)), len(labels))
print(len(os.listdir(TEST_IMG_PATH)), len(sample_submission))

10222 10222
10357 10357


In [66]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
labels = labels[labels['breed'].isin(selected_breed_list)]
labels = labels[labels['breed'].isin(selected_breed_list)]
labels['target'] = 1

group = labels.groupby(by='breed', as_index=False).agg({'id': pd.Series.nunique}).sort_values('id',ascending=False)
labels['rank'] = group['breed']

##labels['rank'] = labels.groupby('breed').rank()['id'] <-- comment out this line

labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)

train = labels_pivot.sample(frac=0.8)
valid = labels_pivot[~labels_pivot['id'].isin(train['id'])]
print(train.shape, valid.shape)

(194, 3) (49, 3)


In [67]:
selected_breed_list = list(labels.groupby('breed')
                           .count().sort_values(by='id', ascending=False)
                           .head(NUM_CLASSES).index)

labels = labels[labels['breed'].isin(selected_breed_list)]
labels['target'] = 1

group = labels.groupby(by='breed', as_index=False).agg({'id': pd.Series.nunique}).sort_values('id',ascending=False)
labels['rank'] = group['breed']

##labels['rank'] = labels.groupby('breed').rank()['id'] <-- comment out this line

labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)

rnd = np.random.random(len(labels))

train_idx = rnd < 0.8
valid_idx = rnd >= 0.8

y_train = labels_pivot[selected_breed_list].values
ytr = y_train[train_idx]
yv = y_train[valid_idx]

print(ytr.shape, yv.shape)

(191, 2) (52, 2)
